This project was created by Brandon Liunoras.
This module goes over how to get song data by artist.
There are other ways to get data such as by album and by track, but this focuses by artist.

In [ ]:
!pip install spotipy

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
# Authentication Credentials
# Project Use Only
# For any replication, you will need own client_id and client_secret
client_credentials = SpotifyClientCredentials(client_id="need own", 
                                              client_secret="need own")

# "sp" is the main function to make api calls
# For all Spotipy References, go to https://spotipy.readthedocs.io/en
sp = spotipy.Spotify(client_credentials_manager = client_credentials)

In [19]:
# In artist_name_search, type the name of the artist that you want to search.

################################################
artist_name_search = ["Bruno Mars"] ################
################################################

artist_lookup = sp.search(artist_name_search)
artist_lookup['tracks']['items'][0]['artists'] # Returns href, id, name, type, and uri of the search.

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/0du5cEVh5yTK9QJze8zA0C'},
  'href': 'https://api.spotify.com/v1/artists/0du5cEVh5yTK9QJze8zA0C',
  'id': '0du5cEVh5yTK9QJze8zA0C',
  'name': 'Bruno Mars',
  'type': 'artist',
  'uri': 'spotify:artist:0du5cEVh5yTK9QJze8zA0C'}]

In [6]:
# To find all of the songs of an artist, the albums of the artist must be pulled
artist_uri = artist_lookup['tracks']['items'][0]['artists'][0]['uri']
artist_name = artist_lookup['tracks']['items'][0]['artists'][0]['name']

print("Corresponding URI:", artist_uri)
print("The artist you have searched is:", artist_name)

Corresponding URI: spotify:artist:0du5cEVh5yTK9QJze8zA0C
The artist you have searched is: Bruno Mars


In [7]:
artist_albums = sp.artist_albums(artist_uri, album_type='album')

# Storing Album Names and URI's in different lists
artist_album_names = []
artist_album_uris = []

for i in range(len(artist_albums['items'])):
    artist_album_names.append(artist_albums['items'][i]['name'])
    artist_album_uris.append(artist_albums['items'][i]['uri'])

print(artist_album_names)
print(artist_album_uris)

# There may be something that says "Couldn't read cache at: .cache"
# This is not an error and the program is working as intended.

['An Evening With Silk Sonic', '24K Magic', 'Unorthodox Jukebox', 'Doo-Wops & Hooligans']
['spotify:album:4VZ7jhV0wHpoNPCB7Vmiml', 'spotify:album:4PgleR09JVnm3zY1fW3XBA', 'spotify:album:58ufpQsJ1DS5kq4hhzQDiI', 'spotify:album:1uyf3l2d4XYwiEqAb7t7fX']


In [8]:
def album_songs(uri):
    album = uri 
    album_dictionary[album] = {}
   
    # Create keys-values of empty lists inside nested dictionary for album
    album_dictionary[album]['album'] = [] 
    album_dictionary[album]['track_number'] = []
    album_dictionary[album]['id'] = []
    album_dictionary[album]['name'] = []
    album_dictionary[album]['uri'] = []
    
    # pull data on album tracks
    tracks = sp.album_tracks(album) 
    for n in range(len(tracks['items'])): 
        album_dictionary[album]['album'].append(artist_album_names[album_count]) 
        album_dictionary[album]['track_number'].append(tracks['items'][n]['track_number'])
        album_dictionary[album]['id'].append(tracks['items'][n]['id'])
        album_dictionary[album]['name'].append(tracks['items'][n]['name'])
        album_dictionary[album]['uri'].append(tracks['items'][n]['uri'])

In [9]:
album_dictionary = {}
album_count = 0
for i in artist_album_uris: # each album
    album_songs(i)
    print(str(artist_album_names[album_count]) + " songs have been added to album_dictionary")
    album_count+=1 # Updates album count once all tracks have been added

An Evening With Silk Sonic songs have been added to album_dictionary
24K Magic songs have been added to album_dictionary
Unorthodox Jukebox songs have been added to album_dictionary
Doo-Wops & Hooligans songs have been added to album_dictionary


In [10]:
def audio_features(album):
    # Add new key-values to store audio features
    album_dictionary[album]['acousticness'] = []
    album_dictionary[album]['danceability'] = []
    album_dictionary[album]['energy'] = []
    album_dictionary[album]['instrumentalness'] = []
    album_dictionary[album]['liveness'] = []
    album_dictionary[album]['loudness'] = []
    album_dictionary[album]['speechiness'] = []
    album_dictionary[album]['tempo'] = []
    album_dictionary[album]['valence'] = []
    album_dictionary[album]['popularity'] = []
    
    track_count = 0
    for track in album_dictionary[album]['uri']:
        # Pulling audio features for each track
        features = sp.audio_features(track)
        
        # Append to relevant key-value
        album_dictionary[album]['acousticness'].append(features[0]['acousticness'])
        album_dictionary[album]['danceability'].append(features[0]['danceability'])
        album_dictionary[album]['energy'].append(features[0]['energy'])
        album_dictionary[album]['instrumentalness'].append(features[0]['instrumentalness'])
        album_dictionary[album]['liveness'].append(features[0]['liveness'])
        album_dictionary[album]['loudness'].append(features[0]['loudness'])
        album_dictionary[album]['speechiness'].append(features[0]['speechiness'])
        album_dictionary[album]['tempo'].append(features[0]['tempo'])
        album_dictionary[album]['valence'].append(features[0]['valence'])
        #popularity is stored elsewhere
        pop = sp.track(track)
        album_dictionary[album]['popularity'].append(pop['popularity'])
        track_count+=1

In [11]:
# Longest part of the read/write process
# This module can cause the most problwems
# Error 429 = too many calls at one time. Give time and try again
# If this does not work, create a new app and get a new client_id and client_secret

import time
import numpy as np
sleep_min = 2
sleep_max = 6
start_time = time.time()
count_request = 0
for i in album_dictionary:
    audio_features(i)
    count_request+=1
    if count_request % 5 == 0:
        print(str(count_request) + " playlists completed")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop Count: {}'.format(count_request))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

In [12]:
# Formulating the dictionary to dataframe form
dic_df = {}
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []
for album in album_dictionary: 
    for feature in album_dictionary[album]:
        dic_df[feature].extend(album_dictionary[album][feature])
        
print('Number of Songs in Dictionary:', len(dic_df['album']))

Number of Songs in Dictionary: 39


In [13]:
import pandas as pd
albums_df = pd.DataFrame.from_dict(dic_df)
albums_df

,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,An Evening With Silk Sonic,1,4K09vJ27xCOreumtSuU6Ao,Silk Sonic Intro,spotify:track:4K09vJ27xCOreumtSuU6Ao,0.09700,0.696,0.553,0.000004,0.6170,-8.702,0.0759,114.993,0.6740,64
1,An Evening With Silk Sonic,2,4pryE6cN2gFL1FVF5fYINl,Leave The Door Open,spotify:track:4pryE6cN2gFL1FVF5fYINl,0.18200,0.586,0.616,0.000000,0.0927,-7.964,0.0324,148.088,0.7190,73
2,An Evening With Silk Sonic,3,7suB6D6uKX5DfPukdGaz0W,Fly As Me,spotify:track:7suB6D6uKX5DfPukdGaz0W,0.00756,0.783,0.627,0.000000,0.0606,-8.521,0.1020,100.025,0.4800,62
3,An Evening With Silk Sonic,4,6jGAh1bFnXt1Muj9zeHveZ,After Last Night (with Thundercat & Bootsy Col...,spotify:track:6jGAh1bFnXt1Muj9zeHveZ,0.02970,0.651,0.703,0.000000,0.0608,-8.958,0.0816,140.051,0.6470,71
4,An Evening With Silk Sonic,5,1oERlssLrpssCAY6Yqqs6c,Smokin Out The Window,spotify:track:1oERlssLrpssCAY6Yqqs6c,0.05580,0.627,0.618,0.000000,0.3510,-8.529,0.0437,82.030,0.8480,67
5,An Evening With Silk Sonic,6,5lka5RUbLVQGO94mKAPMRO,Put On A Smile,spotify:track:5lka5RUbLVQGO94mKAPMRO,0.04710,0.548,0.627,0.000000,0.2850,-8.949,0.0409,143.994,0.4930,60
6,An Evening With Silk Sonic,7,2K6vUau7bnZUamjbRSOOvJ,777,spotify:track:2K6vUau7bnZUamjbRSOOvJ,0.00269,0.836,0.622,0.000566,0.3350,-11.003,0.0946,102.015,0.8920,63
7,An Evening With Silk Sonic,8,3WTWh2WDk4j8GUCGj4xfOd,Skate,spotify:track:3WTWh2WDk4j8GUCGj4xfOd,0.03700,0.708,0.598,0.000000,0.1700,-8.365,0.0291,112.027,0.6980,64
8,An Evening With Silk Sonic,9,2ALh2jqA7KldpHMUHvRomw,Love's Train,spotify:track:2ALh2jqA7KldpHMUHvRomw,0.03960,0.711,0.605,0.000023,0.1040,-9.801,0.0310,140.981,0.8070,62
9,An Evening With Silk Sonic,10,2NqyjfDXy0XfXCSPXMsKzi,Blast Off,spotify:track:2NqyjfDXy0XfXCSPXMsKzi,0.03630,0.482,0.548,0.000002,0.3000,-9.946,0.0373,76.017,0.3460,58


In [14]:
# Removal of duplicates and ensure proper sorting

print('Number of Songs in Original Dictionary:', len(albums_df))

new_albums_df = albums_df.sort_values('popularity', ascending=False)
new_albums_df = albums_df.drop_duplicates('name').sort_index()

print('Number of Songs in New Dictionary:', len(new_albums_df))

Number of Songs in Original Dictionary: 39
Number of Songs in New Dictionary: 39


In [15]:
new_albums_df

,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,An Evening With Silk Sonic,1,4K09vJ27xCOreumtSuU6Ao,Silk Sonic Intro,spotify:track:4K09vJ27xCOreumtSuU6Ao,0.09700,0.696,0.553,0.000004,0.6170,-8.702,0.0759,114.993,0.6740,64
1,An Evening With Silk Sonic,2,4pryE6cN2gFL1FVF5fYINl,Leave The Door Open,spotify:track:4pryE6cN2gFL1FVF5fYINl,0.18200,0.586,0.616,0.000000,0.0927,-7.964,0.0324,148.088,0.7190,73
2,An Evening With Silk Sonic,3,7suB6D6uKX5DfPukdGaz0W,Fly As Me,spotify:track:7suB6D6uKX5DfPukdGaz0W,0.00756,0.783,0.627,0.000000,0.0606,-8.521,0.1020,100.025,0.4800,62
3,An Evening With Silk Sonic,4,6jGAh1bFnXt1Muj9zeHveZ,After Last Night (with Thundercat & Bootsy Col...,spotify:track:6jGAh1bFnXt1Muj9zeHveZ,0.02970,0.651,0.703,0.000000,0.0608,-8.958,0.0816,140.051,0.6470,71
4,An Evening With Silk Sonic,5,1oERlssLrpssCAY6Yqqs6c,Smokin Out The Window,spotify:track:1oERlssLrpssCAY6Yqqs6c,0.05580,0.627,0.618,0.000000,0.3510,-8.529,0.0437,82.030,0.8480,67
5,An Evening With Silk Sonic,6,5lka5RUbLVQGO94mKAPMRO,Put On A Smile,spotify:track:5lka5RUbLVQGO94mKAPMRO,0.04710,0.548,0.627,0.000000,0.2850,-8.949,0.0409,143.994,0.4930,60
6,An Evening With Silk Sonic,7,2K6vUau7bnZUamjbRSOOvJ,777,spotify:track:2K6vUau7bnZUamjbRSOOvJ,0.00269,0.836,0.622,0.000566,0.3350,-11.003,0.0946,102.015,0.8920,63
7,An Evening With Silk Sonic,8,3WTWh2WDk4j8GUCGj4xfOd,Skate,spotify:track:3WTWh2WDk4j8GUCGj4xfOd,0.03700,0.708,0.598,0.000000,0.1700,-8.365,0.0291,112.027,0.6980,64
8,An Evening With Silk Sonic,9,2ALh2jqA7KldpHMUHvRomw,Love's Train,spotify:track:2ALh2jqA7KldpHMUHvRomw,0.03960,0.711,0.605,0.000023,0.1040,-9.801,0.0310,140.981,0.8070,62
9,An Evening With Silk Sonic,10,2NqyjfDXy0XfXCSPXMsKzi,Blast Off,spotify:track:2NqyjfDXy0XfXCSPXMsKzi,0.03630,0.482,0.548,0.000002,0.3000,-9.946,0.0373,76.017,0.3460,58


In [16]:
# Renaming columns for future added columns
new_albums_df.rename(columns={"id": "song_id"}, inplace=True)
new_albums_df.rename(columns={"uri": "song_uri"}, inplace=True)
new_albums_df.rename(columns={"album": "album_name"}, inplace = True)
new_albums_df.rename(columns={"name": "song_name"}, inplace = True)
new_albums_df

,album_name,track_number,song_id,song_name,song_uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,An Evening With Silk Sonic,1,4K09vJ27xCOreumtSuU6Ao,Silk Sonic Intro,spotify:track:4K09vJ27xCOreumtSuU6Ao,0.09700,0.696,0.553,0.000004,0.6170,-8.702,0.0759,114.993,0.6740,64
1,An Evening With Silk Sonic,2,4pryE6cN2gFL1FVF5fYINl,Leave The Door Open,spotify:track:4pryE6cN2gFL1FVF5fYINl,0.18200,0.586,0.616,0.000000,0.0927,-7.964,0.0324,148.088,0.7190,73
2,An Evening With Silk Sonic,3,7suB6D6uKX5DfPukdGaz0W,Fly As Me,spotify:track:7suB6D6uKX5DfPukdGaz0W,0.00756,0.783,0.627,0.000000,0.0606,-8.521,0.1020,100.025,0.4800,62
3,An Evening With Silk Sonic,4,6jGAh1bFnXt1Muj9zeHveZ,After Last Night (with Thundercat & Bootsy Col...,spotify:track:6jGAh1bFnXt1Muj9zeHveZ,0.02970,0.651,0.703,0.000000,0.0608,-8.958,0.0816,140.051,0.6470,71
4,An Evening With Silk Sonic,5,1oERlssLrpssCAY6Yqqs6c,Smokin Out The Window,spotify:track:1oERlssLrpssCAY6Yqqs6c,0.05580,0.627,0.618,0.000000,0.3510,-8.529,0.0437,82.030,0.8480,67
5,An Evening With Silk Sonic,6,5lka5RUbLVQGO94mKAPMRO,Put On A Smile,spotify:track:5lka5RUbLVQGO94mKAPMRO,0.04710,0.548,0.627,0.000000,0.2850,-8.949,0.0409,143.994,0.4930,60
6,An Evening With Silk Sonic,7,2K6vUau7bnZUamjbRSOOvJ,777,spotify:track:2K6vUau7bnZUamjbRSOOvJ,0.00269,0.836,0.622,0.000566,0.3350,-11.003,0.0946,102.015,0.8920,63
7,An Evening With Silk Sonic,8,3WTWh2WDk4j8GUCGj4xfOd,Skate,spotify:track:3WTWh2WDk4j8GUCGj4xfOd,0.03700,0.708,0.598,0.000000,0.1700,-8.365,0.0291,112.027,0.6980,64
8,An Evening With Silk Sonic,9,2ALh2jqA7KldpHMUHvRomw,Love's Train,spotify:track:2ALh2jqA7KldpHMUHvRomw,0.03960,0.711,0.605,0.000023,0.1040,-9.801,0.0310,140.981,0.8070,62
9,An Evening With Silk Sonic,10,2NqyjfDXy0XfXCSPXMsKzi,Blast Off,spotify:track:2NqyjfDXy0XfXCSPXMsKzi,0.03630,0.482,0.548,0.000002,0.3000,-9.946,0.0373,76.017,0.3460,58


In [17]:
# Adding Artist Information for later combining of dataframes
new_albums_df.insert(5, 'artist_name', artist_name)
new_albums_df.insert(6, 'artist_uri', artist_uri)
new_albums_df

,album_name,track_number,song_id,song_name,song_uri,artist_name,artist_uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,An Evening With Silk Sonic,1,4K09vJ27xCOreumtSuU6Ao,Silk Sonic Intro,spotify:track:4K09vJ27xCOreumtSuU6Ao,Bruno Mars,spotify:artist:0du5cEVh5yTK9QJze8zA0C,0.09700,0.696,0.553,0.000004,0.6170,-8.702,0.0759,114.993,0.6740,64
1,An Evening With Silk Sonic,2,4pryE6cN2gFL1FVF5fYINl,Leave The Door Open,spotify:track:4pryE6cN2gFL1FVF5fYINl,Bruno Mars,spotify:artist:0du5cEVh5yTK9QJze8zA0C,0.18200,0.586,0.616,0.000000,0.0927,-7.964,0.0324,148.088,0.7190,73
2,An Evening With Silk Sonic,3,7suB6D6uKX5DfPukdGaz0W,Fly As Me,spotify:track:7suB6D6uKX5DfPukdGaz0W,Bruno Mars,spotify:artist:0du5cEVh5yTK9QJze8zA0C,0.00756,0.783,0.627,0.000000,0.0606,-8.521,0.1020,100.025,0.4800,62
3,An Evening With Silk Sonic,4,6jGAh1bFnXt1Muj9zeHveZ,After Last Night (with Thundercat & Bootsy Col...,spotify:track:6jGAh1bFnXt1Muj9zeHveZ,Bruno Mars,spotify:artist:0du5cEVh5yTK9QJze8zA0C,0.02970,0.651,0.703,0.000000,0.0608,-8.958,0.0816,140.051,0.6470,71
4,An Evening With Silk Sonic,5,1oERlssLrpssCAY6Yqqs6c,Smokin Out The Window,spotify:track:1oERlssLrpssCAY6Yqqs6c,Bruno Mars,spotify:artist:0du5cEVh5yTK9QJze8zA0C,0.05580,0.627,0.618,0.000000,0.3510,-8.529,0.0437,82.030,0.8480,67
5,An Evening With Silk Sonic,6,5lka5RUbLVQGO94mKAPMRO,Put On A Smile,spotify:track:5lka5RUbLVQGO94mKAPMRO,Bruno Mars,spotify:artist:0du5cEVh5yTK9QJze8zA0C,0.04710,0.548,0.627,0.000000,0.2850,-8.949,0.0409,143.994,0.4930,60
6,An Evening With Silk Sonic,7,2K6vUau7bnZUamjbRSOOvJ,777,spotify:track:2K6vUau7bnZUamjbRSOOvJ,Bruno Mars,spotify:artist:0du5cEVh5yTK9QJze8zA0C,0.00269,0.836,0.622,0.000566,0.3350,-11.003,0.0946,102.015,0.8920,63
7,An Evening With Silk Sonic,8,3WTWh2WDk4j8GUCGj4xfOd,Skate,spotify:track:3WTWh2WDk4j8GUCGj4xfOd,Bruno Mars,spotify:artist:0du5cEVh5yTK9QJze8zA0C,0.03700,0.708,0.598,0.000000,0.1700,-8.365,0.0291,112.027,0.6980,64
8,An Evening With Silk Sonic,9,2ALh2jqA7KldpHMUHvRomw,Love's Train,spotify:track:2ALh2jqA7KldpHMUHvRomw,Bruno Mars,spotify:artist:0du5cEVh5yTK9QJze8zA0C,0.03960,0.711,0.605,0.000023,0.1040,-9.801,0.0310,140.981,0.8070,62
9,An Evening With Silk Sonic,10,2NqyjfDXy0XfXCSPXMsKzi,Blast Off,spotify:track:2NqyjfDXy0XfXCSPXMsKzi,Bruno Mars,spotify:artist:0du5cEVh5yTK9QJze8zA0C,0.03630,0.482,0.548,0.000002,0.3000,-9.946,0.0373,76.017,0.3460,58


In [18]:
# Moving the Dataframe to csv for safekeeping
# Rename the csv as "artist_name_songs.csv"
# Insert actual artist name in place of "artist_name"
new_albums_df.to_csv("bruno_mars_songs.csv")